In [250]:
%run download_geographic_refs.ipynb
%run set_up.py 

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import nltk
from nltk.util import ngrams
import re

verbose=True

In [357]:
wbd_path = os.path.join(nhd_dir, 'WBD_National_GDB.gdb')
hu12_parquet = os.path.join(nhd_dir, 'wbd_hu12list.parquet')
basinatlas_path = os.path.join(hydroatlas_dir,  'BasinATLAS_v10.gdb')
basinatlas11_parquet = os.path.join(hydroatlas_dir, 'basinatlas_lev11_idlist.parquet')
geoglows_vpu_path = os.path.join(geoglows_dir, 'vpu-boundaries.gpkg')

In [252]:
#Create a list of NHD HUC12
def create_huc12_list(wbd_path, 
                      out_hu12_parquet,
                      verbose=True):
    if not os.path.exists(out_hu12_parquet):
        if verbose:
            print(f'Generating a list of HUC 12 and saving it to \
            {out_hu12_parquet}')
        wbdhu12_list = gpd.read_file(filename=wbd_path, 
                                     layer='WBDHU12', 
                                     rows=105000,
                                     columns=['huc12'],
                                     ignore_geometry=True)
        wbdhu12_list.to_parquet(out_hu12_parquet)
    else:
        wbdhu12_list = pd.read_parquet(out_hu12_parquet)
    return(wbdhu12_list)

hu12_list = create_huc12_list(wbd_path, hu12_parquet)

In [313]:
#Create a list of PFAF_ID for basins level 11
def create_basinatlas11_list(basinatlas_path, 
                             out_basinatlas11_parquet, 
                             verbose=True):
    if not os.path.exists(out_basinatlas11_parquet):
        if verbose:
            print(f'Generating a list of PFAF ID level 11 and saving it to \
            {out_basinatlas11_parquet}')
            
        basinatlas11_list = gpd.read_file(
            filename=basinatlas_path, 
            layer='BasinATLAS_v10_lev11', 
            columns=['PFAF_ID'],
            rows=1031785,
            ignore_geometry=True).\
        astype(pd.Int64Dtype()).\
        rename(columns={"PFAF_ID": "PFAF_ID11"})
        
        basinatlas11_list.to_parquet(out_basinatlas11_parquet)
    else:
        basinatlas11_list = pd.read_parquet(out_basinatlas11_parquet)
    return(basinatlas11_list)

basinatlas11_list = create_basinatlas11_list(
    basinatlas_path, 
    basinatlas11_parquet)

Generating a list of PFAF ID level 11 and saving it to             D:\WWF_SBTN\BTT_analysis\data\hydroatlas\basinatlas_lev11_idlist.parquet


In [351]:
def _expand_basin_idlist(in_id_list,
                         in_refids_parquet,
                         refids_col,
                         out_id_range):
    """
    Expands a list of basin IDs by extracting IDs at different levels from a reference Parquet file.

    Args:
        in_id_list (list): List of input basin IDs.  Can be integers or strings.
        in_refids_parquet (str): Path to the reference Parquet file.
        refids_col (str): Name of the column in the Parquet file containing the full basin IDs.
        out_id_range (list): List of integer levels to extract.  e.g., [6, 9, 12]

    Returns:
        pandas.DataFrame: DataFrame containing the expanded basin IDs.  The returned DataFrame
                          will have columns named based on `refids_col` and levels in `out_id_range`.
                          The data type of the output columns will match the input type of `in_id_list`.

    Raises:
        TypeError: If the length of input IDs is inconsistent with the length in reference table.
        FileNotFoundError: If the input Parquet file does not exist.
        ValueError:  If `out_id_range` contains values greater than the maximum ID length.
                   Or if an empty DataFrame is returned by filtering.
    """

    id_all_pd = pd.read_parquet(in_refids_parquet)

    # Determine input type and maximum ID length
    in_id_type = type(in_id_list[0])
    if in_id_type == str:
      in_id_len = len(in_id_list[0])
    elif in_id_type == int:
      in_id_len = len(str(in_id_list[0]))
    else:
      raise TypeError("in_id_list must be a list of strings or integers")

    # Determine the reference ID type and length
    first_ref_id = id_all_pd[refids_col].iloc[0] #Get first item in col
    refid_type = type(first_ref_id)

    if refid_type == str:
      refid_len = len(first_ref_id)
    elif pd.api.types.is_integer_dtype(refid_type) or  refid_type == np.int64:
      refid_len = len(str(first_ref_id))
    else:
      raise TypeError(f"{refids_col} must contain strings or integers")


    if in_id_len > refid_len:
        raise ValueError(f"Input ID length ({in_id_len}) is greater than reference ID length ({refid_len})")

    # --- Input Validation on out_id_range ---
    if any(level > refid_len for level in out_id_range):
        raise ValueError(f"out_id_range values cannot exceed reference ID length ({refid_len})")

    # --- Filtering ---
    # Convert to string for consistent prefix matching, then convert back later
    id_all_pd[refids_col] = id_all_pd[refids_col].astype(str)
    in_id_list_str = [str(x) for x in in_id_list]

    id_pd = id_all_pd[id_all_pd[refids_col].str.startswith(tuple(in_id_list_str))].copy()

    if id_pd.empty:
        raise ValueError("No matching IDs found. Check in_id_list and refids_col.")

    # --- Column Expansion ---
    colroot = re.sub(r'[0-9]+', '', refids_col)
    for id_level in out_id_range:
        col_name = f'{colroot}{id_level}'
        if col_name not in id_pd.columns:
            id_pd.loc[:, col_name] = id_pd[refids_col].str[:id_level]
            # Convert back to original type, if necessary
            if in_id_type == int:
                id_pd.loc[:, col_name] = pd.to_numeric(id_pd[col_name],
                                                       errors='raise') 

    # Convert refids_col back to original type
    if refid_type == int:
        id_pd[refids_col] = pd.to_numeric(id_pd[refids_col], 
                                          errors='raise')

    return id_pd

In [343]:
def _format_gdf_tojoin(in_xytab=None, lon_col=None, lat_col=None, 
                       in_vector=None, hull=True):
    #Read xy table
    if in_xytab:
        points_df = pd.read_table(point_locations_path)
        gdf_to_join = gpd.GeoDataFrame(
            points_df,
            geometry=[Point(xy) for xy 
                      in zip(points_df[lon_col], points_df[lat_col])]
        )
    
    #Read vector layer
    if in_vector:
        gdf_to_join = gpd.read_file(in_vector)

    if hull:
        # Create convex hull using union_all() (current recommended method)
        gdf_to_join = gpd.GeoDataFrame(
            geometry=[gdf_to_join.geometry.union_all().convex_hull], 
            crs=gdf_to_join.crs
        )
    return(gdf_to_join)

In [ ]:
def get_matching_NHD_HU(in_wbd_path,
                        in_xytab=None, lon_col=None, lat_col=None,
                        in_vector=None, 
                        in_id_list=None, 
                        in_refids_parquet=None,
                        hull=True, sjoin_predicate='intersects'):

    #If points or polygons are provided ----------------------------------------
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Reach NHD WBD
        wbdhu6 = gpd.read_file(filename=in_wbd_path, 
                               layer='WBDHU6',
                               columns=['huc6']
                              )
    
        #Spatially join to hydrologic units
        matched_nhd = gpd.sjoin(gdf_to_join.to_crs(crs=wbdhu6.crs), 
                               wbdhu6, 
                               how='left', 
                               predicate=sjoin_predicate)
        in_id_list = matched_nhd.huc6.values.tolist()
        
    huc_pd = _expand_basin_idlist(
        in_id_list, 
        in_refids_parquet, 
        refids_col='huc12', 
        out_id_range=range(2, 14, 2))

    return(huc_pd)

test_pts_path = os.path.join(datdir, 'test_gages', 'test_gages.shp')
test_huc_pd = get_matching_NHD_HU(
    in_wbd_path=wbd_path,
    in_vector=test_pts_path,
    in_refids_parquet=hu12_parquet,
    hull=True,
    sjoin_predicate='intersects'
)
print(test_huc_pd)

#in_id_list = in_umrb_huc4s = [f'07{str(i).zfill(2)}' for i in range(2,15)]

In [ ]:
def get_matching_hydrobasin(in_basinatlas_path,
                            in_xytab=None, lon_col=None, lat_col=None,
                            in_vector=None, 
                            in_id_list=None, 
                            in_refids_parquet=None,
                            hull=True, sjoin_predicate='intersects'):
    #If points or polygons are provided ----------------------------------------
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Reach NHD WBD
        bas_lev6 = gpd.read_file(filename=in_basinatlas_path, 
                                 layer='BasinATLAS_v10_lev06',
                                 columns=['PFAF_ID']
                                ).rename(columns={"PFAF_ID": "PFAF_ID6"})
    
        #Spatially join to hydrologic units
        matched_bas = gpd.sjoin(gdf_to_join.to_crs(crs=bas_lev6.crs), 
                               bas_lev6, 
                               how='left', 
                               predicate=sjoin_predicate)
        in_id_list = matched_bas.PFAF_ID6.tolist()

    pfaf_pd = _expand_basin_idlist(
        in_id_list, 
        in_refids_parquet, 
        refids_col='PFAF_ID11', 
        out_id_range=range(3, 12))

    return(pfaf_pd)

test_pfaf_pd = get_matching_hydrobasin(
    in_basinatlas_path=basinatlas_path,
    in_vector=test_pts_path,
    #in_id_list=None, 
    in_refids_parquet=basinatlas11_parquet,
    hull=True,
    sjoin_predicate='intersects'
)
print(test_pfaf_pd)

test_pfaf_pd_idlist = get_matching_hydrobasin(
    in_basinatlas_path=basinatlas_path,
    in_id_list=[742873, 742875, 742876], 
    in_refids_parquet=basinatlas11_parquet
)
print(test_pfaf_pd_idlist)


In [361]:
def get_geoglows_vpu(in_geoglows_vpu_path,
                     in_xytab=None, lon_col=None, lat_col=None,
                     in_vector=None, 
                     in_id_list=None, 
                     in_refids_parquet=None,
                     hull=True, sjoin_predicate='intersects'):
    
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Reach NHD WBD
        vpus = gpd.read_file(filename=in_geoglows_vpu_path, 
                                 layer='vpu-boundaries',
                                 columns=['VPU']
                                )
    
        #Spatially join to hydrologic units
        matched_vpus = gpd.sjoin(gdf_to_join.to_crs(crs=vpus.crs), 
                                 vpus, 
                                 how='left', 
                                 predicate=sjoin_predicate)
        vpu_list = matched_vpus.VPU.tolist()

    return(vpu_list)

test_vpu_list = get_geoglows_vpu(
    in_geoglows_vpu_path=geoglows_vpu_path,
    in_vector=test_pts_path,
    hull=True,
    sjoin_predicate='intersects'
)
print(test_vpu_list)

['714', '709']


In [ ]:
def load_spatial_data(point_locations_path, nhd_path, hydrobasins_path, admin_units_path):
    # Convert your points to GeoDataFrame

    
    # Load boundary files
    nhd_gdf = gpd.read_file(nhd_path)
    hydrobasins_gdf = gpd.read_file(hydrobasins_path)
    admin_gdf = gpd.read_file(admin_units_path)
    
    return points_gdf, nhd_gdf, hydrobasins_gdf, admin_gdf

In [ ]:
# 2. Perform spatial joins to get catchment information
def get_catchment_info(point_gdf, nhd_gdf, hydrobasins_gdf, admin_gdf):
    # Spatial joins
    points_nhd = gpd.sjoin(point_gdf, nhd_gdf, how='left', op='within')
    points_hydrobasins = gpd.sjoin(point_gdf, hydrobasins_gdf, how='left', op='within')
    points_admin = gpd.sjoin(point_gdf, admin_gdf, how='left', op='within')
    
    # Collect all relevant names
    location_names = set()
    
    # Add names from each source (adjust column names as needed)
    name_columns = {
        'nhd': ['BASIN_NAME', 'RIVER_NAME'],
        'hydrobasins': ['HYBAS_NAME'],
        'admin': ['COUNTY_NAME', 'STATE_NAME', 'COUNTRY_NAME']
    }
    
    for df, cols in zip([points_nhd, points_hydrobasins, points_admin], 
                       name_columns.values()):
        for col in cols:
            if col in df.columns:
                location_names.update(df[col].dropna().unique())
    
    return location_names

In [ ]:
# 3. Process OpenAlex records
def process_openalex_records(openalex_df, location_names):
    # Function to generate n-grams
    def get_ngrams(text, n_range=(1, 3)):
        if pd.isna(text):
            return set()
        
        text = str(text).lower()
        tokens = nltk.word_tokenize(text)
        all_ngrams = set()
        
        for n in range(n_range[0], n_range[1] + 1):
            text_ngrams = set(' '.join(gram) for gram in ngrams(tokens, n))
            all_ngrams.update(text_ngrams)
            
        return all_ngrams

    # Convert location names to lowercase for matching
    location_names = set(name.lower() for name in location_names)
    
    # Process each record
    matched_records = []
    
    for _, record in openalex_df.iterrows():
        # Combine all text fields
        text_fields = [
            str(record.get('title', '')),
            str(record.get('abstract', '')),
            str(record.get('keywords', ''))
        ]
        
        combined_text = ' '.join(text_fields).lower()
        
        # Generate n-grams from the combined text
        record_ngrams = get_ngrams(' '.join(text_fields))
        
        # Check for matches
        if any(location in combined_text for location in location_names) or \
           any(location in record_ngrams for location in location_names):
            matched_records.append(record)
    
    return pd.DataFrame(matched_records)

In [ ]:
# Main execution
def main():
    # Load your data (replace with actual file paths)
    points_gdf, nhd_gdf, hydrobasins_gdf, admin_gdf = load_spatial_data(
        'points.csv',
        'nhd.shp',
        'hydrobasins.shp',
        'admin_units.shp'
    )
    
    # Get catchment information
    location_names = get_catchment_info(points_gdf, nhd_gdf, hydrobasins_gdf, admin_gdf)
    
    # Load and process OpenAlex records
    openalex_df = pd.read_csv('openalex_records.csv')  # Replace with actual file path
    
    # Filter records based on location names
    matched_records = process_openalex_records(openalex_df, location_names)
    
    # Save results
    matched_records.to_csv('matched_records.csv', index=False)
    